<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
%load_ext Cython

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file

In [ ]:
import numpy as np
import random
import sys
import zipfile

In [ ]:
localfile = "462_ruby_716.zip"
zipfile = zipfile.ZipFile(localfile, 'r')
file = zipfile.open('kazeno_matasaburo.txt', 'r')
bindata = file.read()
text= bindata.decode('shift_jis')

In [ ]:
'''
path = './null.txt'
bindata = open(path, "rb").read()
text = bindata.decode("shift_jis")
'''

In [ ]:
print('Size of text: ', len(text))

In [ ]:
chars = sorted(list(set(text)))
chars

In [ ]:
print('Total chars:', len(chars))

In [ ]:
#順引き、逆引きの辞書を作る
char_indices = dict((c,i) for i,c  in enumerate(chars))
indices_char = dict((i,c) for i,c in enumerate(chars))

In [ ]:
# cut the text  in sentence
maxlen = 40

step = 3
sentences =[]
next_chars = []

In [ ]:
for i in range(0,len(text) - maxlen, step):
    sentences.append(text[i:i + maxlen])
    next_chars.append(text[i+maxlen])

In [ ]:
sentences

In [ ]:
next_chars

In [ ]:
print('Number of sentences:', len(sentences))

In [ ]:
#テキストのベクトル化
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [ ]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i,char_indices[next_chars[i]]] = 1

In [ ]:
#モデルを定義する
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))

In [ ]:
model.add(Dense(len(chars))) #全てのセルを使うニューラルネットワークを作る

In [ ]:
model.add(Activation('softmax')) #活性化関数

In [ ]:
optimizer = RMSprop(lr=0.01) #学習率の指定

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
def sample(preds, temperture=1.0):
    preds = np.array(preds).astype('float64')
    preds = np.log(preds)/temperture
    exp_preds = np.exp(preds)
    preds = exp_preds/ np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
'''
for iteration in range(1,60):
    print()
    print('-' * 50)
    print('繰り返し：', iteration)
    model.fit(X, y, batch_size=128, epochs=1)
    
    start_index = random.randint(0, len(text)-maxlen-1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('-----diversity', diversity)
    
        generated = ''
        sentence = text[start_index: start_index+ maxlen]
        generated += sentence
        print('----- Seedを生成しました：”' + sentence + '”')
        sys.stdout.write(generated)
    
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.
            
            preds = model.predict(x, verbose=9)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
        
            generated += next_char
            sentence = sentence[1:] + next_char
        
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
'''

In [3]:
%%cython
def generate():
    for iteration in range(1,60):
        print()
        print('-' * 50)
        print('繰り返し：', iteration)
        model.fit(X, y, batch_size=128, epochs=1)
    
        start_index = random.randint(0, len(text)-maxlen-1)

        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print()
            print('-----diversity', diversity)
    
            generated = ''
            sentence = text[start_index: start_index+ maxlen]
            generated += sentence
            print('----- Seedを生成しました：”' + sentence + '”')
            sys.stdout.write(generated)
    
            for i in range(400):
                x = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x[0, t, char_indices[char]] = 1.
            
                preds = model.predict(x, verbose=9)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]
        
                generated += next_char
                sentence = sentence[1:] + next_char
        
                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()


Error compiling Cython file:
------------------------------------------------------------
...
def generate():
    for iteration in range(1,60):
        print()
        print('-' * 50)
        print('繰り返し：', iteration)
        model.fit(X, y, batch_size=128, epochs=1)
       ^
------------------------------------------------------------

/Users/yamaguchishota/.ipython/cython/_cython_magic_6d533993bd64345266a27b6f5f9a525f.pyx:6:8: undeclared name not builtin: model

Error compiling Cython file:
------------------------------------------------------------
...
def generate():
    for iteration in range(1,60):
        print()
        print('-' * 50)
        print('繰り返し：', iteration)
        model.fit(X, y, batch_size=128, epochs=1)
                 ^
------------------------------------------------------------

/Users/yamaguchishota/.ipython/cython/_cython_magic_6d533993bd64345266a27b6f5f9a525f.pyx:6:18: undeclared name not builtin: X

Error compiling Cython file:
-------------------------

TypeError: object of type 'NoneType' has no len()